In [ ]:
%matplotlib inline

import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()


utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation


network_name = "3_hat_train_6"
dcn = dce.load_network_from_config(network_name)
dataset = dcn.load_training_dataset()

train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')
train_config = utils.getDictFromYamlFilename(train_config_file)
dataset._setup_data_load_types()
dataset.set_parameters_from_training_config(train_config)

dcn.eval()

scene_name_a = dataset.get_random_scene_name()
img_a_idx = dataset.get_random_image_index(scene_name_a)
rgb_a, _, _, _ = dataset.get_rgbd_mask_pose(scene_name_a, img_a_idx)

scene_name_b = dataset.get_random_scene_name()
img_b_idx = dataset.get_random_image_index(scene_name_b)
rgb_b, _, _, _ = dataset.get_rgbd_mask_pose(scene_name_b, img_b_idx)

rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)
rgb_b_tensor = dataset.rgb_image_to_tensor(rgb_b)

# these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
res_a = dcn.forward_single_image_tensor(rgb_a_tensor)
res_b = dcn.forward_single_image_tensor(rgb_b_tensor)

In [ ]:
print res_a.data.shape

# put in N, C, H, W
res_a_shaped = res_a.permute(2,0,1).unsqueeze(0)
print res_a_shaped.shape

In [ ]:
import torch.nn as nn

# pool of square window of size=3, stride=2
# AvgPool2d wants: N, C, H, W
m = nn.AvgPool2d(8, stride=8)
# pool of non-square window
input = Variable(torch.randn(20, 16, 48, 32))
output = m(input)
print  output.data.shape

In [ ]:
output = m(res_a_shaped)
print output.shape
output_unshaped = output.squeeze(0).permute(1,2,0)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(res_a.data.cpu().numpy())
plt.show()

plt.imshow(output_unshaped.data.cpu().numpy())
plt.show()

In [ ]:
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated

class dcn_discriminator(nn.Module):
    def __init__(self, num_classes):
        super(dcn_discriminator, self).__init__()
        self.avg_pool = nn.AvgPool2d(8, stride=8)
        self.r18 = resnet_dilated.Resnet18_8s(num_classes=num_classes)
        # 8, 10 is hardcoded for 640 x 480 at moment,
        # would be easy to calc general param
        self.fc = nn.Linear(num_classes*8*10, num_classes)
        
    def forward(self, x, HWC_shape=False):
        """
        This forward() directly takes "res_a" out of our DCN (dense descriptor network)
        and sends it to a 1 x 3 Tensor of one-hot encodings for class prediction
        """
        if HWC_shape:
            x = x.permute(2,0,1).unsqueeze(0)
        x = self.avg_pool(x)
        x = self.r18.resnet18_8s(x)
        x = x.view(-1)
        x = self.fc(x).unsqueeze(0)
        return x
    
# currently 6 because using with 6 hats dataset
# but should automate this easily
dcn_d = dcn_discriminator(num_classes=6).cuda()

In [ ]:
predictions = dcn_d(res_a, HWC_shape=True)

#predictions[0,1] = 0
#predictions[0,2] = 0
print predictions

criterion = nn.CrossEntropyLoss()

from torch.autograd import Variable

label = Variable(torch.LongTensor([0]), requires_grad=False).cuda()
d_loss = criterion(predictions, label)
print d_loss

label = Variable(torch.LongTensor([1]), requires_grad=False).cuda()
d_loss = criterion(predictions, label)
print d_loss

label = Variable(torch.LongTensor([2]), requires_grad=False).cuda()
d_loss = criterion(predictions, label)
print d_loss

softmax = nn.LogSoftmax()
print softmax(predictions)

In [ ]:
import torch.optim as optim
d_optimizer = optim.Adam(dcn_d.parameters(), lr=1.0e-4, weight_decay=1.0e-4)

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                          shuffle=True, num_workers=10, drop_last=True)

In [ ]:
for i, data in enumerate(data_loader, 0):
    print i
    
    match_type, \
    img_a, img_b, \
    matches_a, matches_b, \
    masked_non_matches_a, masked_non_matches_b, \
    background_non_matches_a, background_non_matches_b, \
    blind_non_matches_a, blind_non_matches_b, \
    metadata = data
    
    if (match_type == -1).all():
        print "\n empty data, continuing \n"
        continue
    
    label = metadata["object_id_int"]
    
    img_a = Variable(img_a.cuda(), requires_grad=False)
    img_b = Variable(img_b.cuda(), requires_grad=False)
    
    image_a_pred = dcn.forward(img_a)
    image_b_pred = dcn.forward(img_b) 
    
    d_optimizer.zero_grad()
    
    predictions_a = dcn_d(image_a_pred)
    predictions_b = dcn_d(image_b_pred)
    
    label = Variable(label, requires_grad=False).cuda()
    d_loss = criterion(predictions_a, label) + criterion(predictions_b, label)
    
    d_loss.backward()
    d_optimizer.step()
    
    print d_loss.data[0], predictions_a, predictions_b